<a href="https://colab.research.google.com/github/bodorcy/hazifeladatok/blob/main/ml_2_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Egyszerű döntési szabályok

A statisztikai elemzések és vizualizációk definícióit, illetve hogy miért szükségesek ezek a gépi tanuláshoz, olvasd el a kapcsolodó [előadás olvasóleckében](http://www.inf.u-szeged.hu/~rfarkas/ML20/baseline.html)!

Ebben a leckében a [student performance](https://archive.ics.uci.edu/ml/datasets/student+performance#) adatbázist fogjuk használni. Ebben egy középiskolai kurzus diákjai adták meg adataikat és azt akarjuk predikálni, hogy sikeresen el fogják-e tudni végezni a kurzust.

In [ ]:
import pandas as pd

In [ ]:
# töltsük be az adatbázist
df = pd.read_csv("https://raw.githubusercontent.com/arunk13/MSDA-Assignments/master/IS607Fall2015/Assignment3/student-por.csv")
df.head() # nézzünk rá!

valami nagyon nem stimmel...

A read_csv vesszőt vár oszlopelválasztó karakternek, itt meg pontosvessző van (ezért egy oszlopnak tekintette az egész sort).


[`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) dokumentáció

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/arunk13/MSDA-Assignments/master/IS607Fall2015/Assignment3/student-por.csv", sep=';') # sep argumentummal megadhatjuk az elválasztó karaktert
df.head() # mindjárt szebben néz ki :)

In [ ]:
df.shape # adatbázis méretei

In [ ]:
df.columns # oszlopok nevei

## Jellemzők/változók leíró statisztikái

In [ ]:
df.Fjob

In [ ]:
# diszkrét változó értékeinek gyakorisága
df.Fjob.value_counts() # Fjob a diák apjának foglalkozása

In [ ]:
# egy folytonos változó statisztikái
print('Min:', df.age.min())
print('Max:', df.age.max())
print('Átlag:', df.age.mean())
print('Medián:', df.age.median())
print('Szórás:', df.age.std())

In [ ]:
# A describe() az összes folytonos jellemző alapstatisztikáiról ad egy táblázatot (eredménye egy DataFrame)
df.describe()

## Vizualizáció pythonban

A leggyorsabban úgy szerezhetünk az adatbázisunkról benyomásokat, ha vizualizáljuk az egyes változókat, illetve az egyes jellemzők és a célváltozó közötti összefüggéseket. A python notebookok egyből, helyben megjelenítik az ábrákat, ami az adatelemzést igen gyorssá és interaktívvá teszi.

Bővebben a python adatvizualizációról [itt](https://pandas.pydata.org/pandas-docs/stable/visualization.html) olvashatsz.

In [ ]:
# kategórikus változó hisztogramja
df.Fjob.hist()

In [ ]:
df.sex.value_counts()

In [ ]:
# itt egy tortadiagrammon jelenítjük meg a diszkrét változó eloszlását
df.sex.value_counts().plot.pie(figsize=(6, 6)) # figsize-at nagyobb képeket is készíthetünk

In [ ]:
# folytonos változó hisztogramja
df.age.hist()

In [ ]:
# a hist() alapértelmezettként 10 egyenlő méretű intervallumra osztja az értékkészletet
# jelen esetben 8 értékünk van csak, ésszerű 8 intervallumra osztani
df.age.hist(bins=8) # bins az intervallumok száma

In [ ]:
# a vizualizációt kedvünkre testreszabhatjuk!
df.age.hist(bins=8, color='c', edgecolor='k', alpha=0.65) # itt például az oszlopok színét, keretét és átlátszóságát állítjuk be

In [ ]:
# ha simán a plot()-ot hívjuk meg numerikus jellemzőre, az mint idősor rajzolja ki
df.age.plot()

### Boxplot
a boxplot folytonos változó jellemzésére egy hasznos eszköz,  
[lásd](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html)

In [ ]:
df.age.plot.box()

In [ ]:
# összes folytonos változó boxplotja egymás mellett
df.loc[:,'G1':'G3'].plot.box(figsize=(10,5)) # a három teszteredményt szeretnénk egymás mellett látni

## Konstans baseline

Legyen a predikciós feladatunk most egy osztályozási feladat, ahol a két osztályunk (`G3` a záró teszten elért pontszám):

*megfelelt* ha `G3>=10` és

*nem felelt meg* ha `G3<10`

In [ ]:
df.G3>=10

In [ ]:
# vegyünk fel egy új oszlopot az osztályozási feladatnak
df['megfelelt'] = df.G3>=10
df.tail()

In [ ]:
df.megfelelt.value_counts()

Az adatbázisban jóval több megfelelt címkéjű egyedet láttunk. Ezért a leggyakoribb osztály (most frequent class) baseline mindig a megfelelt=True címkét fogja predikálni.

Értékeljük ki ezt az egyszerű döntési szabályt!
Osztályozási feladatokra használhatjuk a találati arányt (accuracy) mint kiértékelési metrika:

In [ ]:
print("accuracy:", df.megfelelt.value_counts()[True] / len(df) )

Ha regressziós feladatként fogalmazzuk meg, azaz a végső teszt pontszámát akarjuk megjósolni, akkor a baseline lehet a tanító adatbázison a célváltozó átlaga.

In [ ]:
df.G3.mean() # G3 a folytonos célváltozó, átlaga a konstans dummy predikció:

In [ ]:
# vizualizáljuk ezt a baselinet!
# ha egy ábrára több dolgot akarunk kirajzolni a matplotlib.pyplot-hoz kellhet visszanyúlni
import matplotlib.pyplot as plt

plt.hist(df.G3, bins=20, color='c', edgecolor='k', alpha=0.65)
# ax utasítások "rárajzolnak" a meglévő plotra. vline egy függőleges vonal.
plt.axvline(df.G3.mean(), color='k', linestyle='dashed', linewidth=1) # ... aminek a színét, szaggatottságát és vastagságát is formázzuk

In [ ]:
pred = df.G3.mean()
# regressziónál a kiértékelési metrika lehet az eltérések (predikciós hiba) abszolútértékének átlaga (Mean Absolute Error, MAE)
print('MAE:', (df.G3 - pred).abs().mean())

In [ ]:
(df.G3 - pred).abs()

## Döntési szabály egyetlen jellemző alapján

Válaszzunk ki egy jellemzőt és írjunk ("kézzel") egyszerű döntési szabályt, ami ezen egyetlen jellemző érétke alapján hoz döntést.


### Osztályozási döntés kategórikus jellemző alapján

Két diszkrét változó közti kapcsolatot könnyen megérthetjük a kereszttáblájukból, ami a két jellemző egyes értékeinek együttes előfordulásának gyakoriságát tartalmazza:

In [ ]:
# két diszkrét változó értékeinek együttelőfordulási gyakoriságai
pd.crosstab(df.higher, df.Fjob)

In [ ]:
# vizualizálhatjuk is a kereszttáblát
pd.crosstab(df.Fjob, df.megfelelt).plot(kind='bar', stacked=False)

In [ ]:
# legyen a döntési szabályunk: HA XXX==YY AKKOR false EGYÉBKÉNT true
pred = df.higher == "yes"

In [ ]:
print("accuracy:", (pred == df.megfelelt).sum() / len(df))

### Osztályozási szabály folytonos változó alapján

Ha osztályozási döntést akarunk hozni egy folytonos jellemző alapján akkor egy folytonos és egy diszkrét valószínűségi változó kapcsolatának elemzésére van szükség.

Ha egy diszkrét változó különböző értékeire külön-külöm szeretnénk statisztikákat számolni, csoportosíthatjuk a rekordokat a változó értékei szerint:

In [ ]:
df.groupby('megfelelt').mean(numeric_only=True) # megfelelt változó egyes értékeire csoportosított elemek csoportátlaga

In [ ]:
df.groupby(['megfelelt','Dalc']).mean(numeric_only=True) # egyszerre több változó szerint is csoportosíthatunk (Descartes szorzat)

In [ ]:
# egy lehetőség a diszkrét változó értékeire bontani az adathalmazt
# majd a részhalmazokra számított hisztogramokat megjeleníteni
df.Dalc.hist(bins=10, by=df.megfelelt) # a by a groupby (csoportosítás) rövid megfelelője

In [ ]:
# A két hisztogramot egyetlen ábrára is elhelyezzhetjük
# Szűrjük le a megfelelt diszkrét változó értékeire az egyedeket.
# A két részhalmazban kirajzoljuk az age hisztogramját.
plt.hist(df.Dalc[df.megfelelt==True], bins=5, alpha=0.5, label='megfelelt') # alpha az áttetszőséget jelenti
plt.hist(df.Dalc[df.megfelelt==False], bins=5, alpha=0.5, label='nem felelt meg')
plt.legend(loc='upper right') # színek jelentésének magyarázata az ábrára

In [ ]:
# egy másik lehetőség, hogy a diszkrét változó értékei szerinti csoportokat boxploton ábrázoljuk
df.boxplot(column='age', by='megfelelt') # a diszkrét változó értékei szerint csoportosított boxok

In [ ]:
# legyen a döntési szabályunk: HA Dalc > 4 AKKOR false EGYÉBKÉNT true
pred = ~(df.Dalc>4)
print("accuracy:", (pred == df.megfelelt).sum() / len(df))

### Regressziós szabály diszkrét változó alapján

Ha regressziós döntést akarunk hozni egy diszkrét jellemző alapján akkor is egy folytonos és egy diszkrét valószínűségi változó kapcsolatának elemzésére van szükség.

In [ ]:
df.groupby('school').G3.mean()

In [ ]:
pred = [ df.groupby('school').G3.mean()[x]  for x in df['school'] ]
pred

In [ ]:
print('MAE:', (df.G3 - pred).abs().mean() )

### Regressziós szabály folytonos változó alapján

Ebben az adatbázisban kevés az igazi folytonos változó, ezért térjünk vissza a [survey adatbázisra](https://stat.ethz.ch/R-manual/R-devel/library/MASS/html/survey.html).

In [ ]:
df_survey = pd.read_csv("https://raw.github.com/vincentarelbundock/Rdatasets/master/csv/MASS/survey.csv")
df_survey.head()

In [ ]:
# scatter plot
df_survey.plot.scatter('Wr.Hnd','Height') # író kéz mérete és magasság kapcsolata

In [ ]:
# összes folytonos változó páronkénti scatter plotja
from pandas.plotting import scatter_matrix
scatter_matrix(df_survey, alpha=0.5, figsize=(10, 10), diagonal='kde')

In [ ]:
# Folytonos változók páronkénti korrelációjának kiszámítása
df_survey[["Wr.Hnd", "NW.Hnd", "Age", "Height"]].corr()

Figyeljük meg, hogy az iró és nem író kéz méretei közt igen erős (0.948), míg a kézméret és testmagasság közt kevésbé erős korreláció (~0.6) figyelhető meg.

Értékeljük ki és hasonlítsuk össze a két baseline megoldásunkat!

In [ ]:
# konstans baseline
pred = df_survey['Wr.Hnd'].mean()
print('MAE:', (df_survey['Wr.Hnd'] - pred).abs().mean())

In [ ]:
(df_survey.Height / df_survey['Wr.Hnd'])

In [ ]:
# ha a Wr.Hnd megjóslása lenne a regressziós feladat, akkor egy baseline predikciós szabály lehetne a magássából becslés:
ratio = (df_survey.Height / df_survey['Wr.Hnd']).mean()
pred = df_survey.Height / ratio
print('MAE:', (df_survey['Wr.Hnd'] - pred).abs().mean())

In [ ]:
ratio

# sklearn

Számtalan gépi tanulási könyvtár létezik, de mi ezen a kurzuson a [Scikit-learn, másnéven sklearn](http://scikit-learn.org/stable/index.html) python csomagot fogjuk használni.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/arunk13/MSDA-Assignments/master/IS607Fall2015/Assignment3/student-por.csv", sep=';')
X = df[["age", 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',]]

### Osztályozási baselineok

In [ ]:
y=df.G3>=10 #diszkrét célváltozó

In [ ]:
from sklearn.dummy import DummyClassifier
# baseline classifier
#strategies : {“most_frequent”, “prior”, “stratified”, “uniform”, “constant”}
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)

In [ ]:
dummy_preds = dummy_clf.predict(X)
dummy_preds

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, dummy_preds)

### Regressziós baselineok

In [ ]:
y = df["G3"]

In [ ]:
from sklearn.dummy import DummyRegressor

In [ ]:
# baseline regressor
# stragéiák amik közül választhatunk: {“mean”, “median”, “quantile”, “constant”}
dummy_reg = DummyRegressor(strategy="mean")
dummy_reg.fit(X, y)
pred_y = dummy_reg.predict(X)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, pred_y)

# Jellemző kiválasztás

Léteznek olyan statisztikák, amelyek a jellemzőket értékelik (pontozzák) a szerint, hogy mennyi információt szolgáltat a célváltozóra. Ezek segítségével kiválaszthatunk (feature selection) néhány erős jellemzőt, amik felett



*   kézi szabályrendszert írunk vagy
*   gépi tanuló megoldásnak segítünk a zajos/haszontalan jellemzőktől megszabadulni.



In [ ]:
y=df.G3>=10 #diszkrét célváltozó
from sklearn.preprocessing import add_dummy_feature
X_noisy = add_dummy_feature(X,value=1)
print(X_noisy[:3])

In [ ]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, SelectPercentile

In [ ]:
vt = VarianceThreshold(threshold=0.1) #csak a feature értékeit nézi (a célváltozót nem), ezeket unsupervised feature selection-nek is hívják
x_filtered = vt.fit_transform(X_noisy)
print(x_filtered[:3])

In [ ]:
sb = SelectKBest(chi2, k=2) #a feature hatását a célváltozóra számszerűsíti, ezeket supervised feature selection-nek is hívják
x_k_best=sb.fit_transform(X,y)
print(x_k_best[:3])

In [ ]:
sb.get_feature_names_out()

In [ ]:
sp = SelectPercentile(chi2, percentile=50)
x_k_best=sp.fit_transform(X,y)
print(x_k_best[:3])

Részletesebben: https://www.geeksforgeeks.org/chi-square-test-for-feature-selection-mathematical-explanation/

In [ ]:
# Ha regressziós feladat
y = df["G3"]
from sklearn.feature_selection import f_regression
sb = SelectKBest(f_regression, k=3)

In [ ]:
x_k_best=sb.fit_transform(X, y)
x_k_best[:3]

In [ ]:
sb.get_feature_names_out()